# Description 
This script scrapes the stoiximan webpage. It provides a dataframe of all the upcoming EPL matches that stoiximan provides together with the odds of the Betting company. It then joins them with the 'GatheredData/eplGMTStdTimeFiles/epl-2023-GMTStandardTime.csv' file and makes a 'Newfixtures.csv' file. In the end it runs all the needed scripts so that the predictor runs correctly for each round.

In [1]:
import requests 
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import datetime

url = 'https://en.stoiximan.gr/sport/soccer/england/premier-league/1/'
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

In [2]:
# A function to change team names as depicted in Stoiximan in the way they are depicted in the files
def replace_names(string):
    old_names = ['Fulham FC', 'Everton FC', 'Leeds United', 'Southampton FC', 'West Ham United', 'Arsenal FC', 'AFC Bournemouth', 'Crystal Palace FC', 'Liverpool FC', 'Brighton & Hove Albion', 'Chelsea FC', 'Tottenham Hotspur', 'Brentford FC', 'Burnley FC', 'Luton Town']
    new_names = ['Fulham', 'Everton', 'Leeds', 'Southampton', 'West Ham', 'Arsenal', 'Bournemouth', 'Crystal Palace', 'Liverpool', 'Brighton', 'Chelsea', 'Tottenham', 'Brentford', 'Burnley', 'Luton']
    
    for i in range(len(old_names)):
        string = string.replace(old_names[i], new_names[i])
    return string

In [3]:

soup = soup.body
script = soup.find_all("script")
script = script[0]

print(script)

<script>window["initial_state"]={"data":{"leagueStandings":[],"leagueGrouping":true,"sport":{"id":"FOOT","name":"Soccer","region":"England","url":"/sport/soccer/"},"parentMarkets":[{"id":"0","name":"Matches","selected":true,"markets":[{"id":"0","name":"Main","selected":true,"type":"Main"},{"id":"1","name":"Double chance"},{"id":"2","name":"Half Time"},{"id":"3","name":"Asian Lines"}]},{"id":"4","name":"Long term bets","type":"LongtermBets"}],"markets":[],"blocks":[{"name":"England - Premier League","shortName":"Premier League","id":"1r","url":"/sport/soccer/england/premier-league/1/","headers":[{"marketTypes":["MR12","MRES"],"columns":[{"columnSpan":1,"name":"1"},{"columnSpan":1,"name":"X"},{"columnSpan":1,"name":"2"}]},{"marketTypes":["HCTG"],"columns":[{"columnSpan":1,"name":"OVER/UNDER"}]},{"marketTypes":["BTSC"],"columns":[{"columnSpan":1,"name":"GG/NG"}]}],"layout":2,"events":[{"stats":[{"url":"https://s5.sir.sportradar.com/stoiximan/en/match/41763007","providerId":1}],"sportId":"

In [4]:
Odds = []
Dates = []
HomeTeams = []
AwayTeams = []

print(script)
    
# Define the regular expression pattern
pattern = r'<script>(.*?)<\/script>'

# Use re.findall to find all matches
matches = re.findall(pattern, str(script), re.DOTALL)

if matches:
    # Extract the content between script tags
    content_between_scripts = matches[0]
    content_between_scripts= content_between_scripts.split('window["initial_state"]=')[1].strip()
    #print(content_between_scripts)
else:
    print('No match found.')
    
d = json.loads(content_between_scripts)


for match in d['data']['blocks'][0]['events']:
    teams = match['shortName'].split('-')
    HomeTeams.append(replace_names(teams[0]))
    AwayTeams.append(replace_names(teams[1]))
    Dates.append(datetime.datetime.fromtimestamp(match['startTime']/1000.0).strftime('%d/%m/%Y'))
    for x in match['markets']:
        for y in x['selections']:
            Odds.append(y['price'])
            


rows = []
while Odds:
    rows.append(Odds[:7])
    Odds = Odds[7:]

print(rows)
print(Dates)

<script>window["initial_state"]={"data":{"leagueStandings":[],"leagueGrouping":true,"sport":{"id":"FOOT","name":"Soccer","region":"England","url":"/sport/soccer/"},"parentMarkets":[{"id":"0","name":"Matches","selected":true,"markets":[{"id":"0","name":"Main","selected":true,"type":"Main"},{"id":"1","name":"Double chance"},{"id":"2","name":"Half Time"},{"id":"3","name":"Asian Lines"}]},{"id":"4","name":"Long term bets","type":"LongtermBets"}],"markets":[],"blocks":[{"name":"England - Premier League","shortName":"Premier League","id":"1r","url":"/sport/soccer/england/premier-league/1/","headers":[{"marketTypes":["MR12","MRES"],"columns":[{"columnSpan":1,"name":"1"},{"columnSpan":1,"name":"X"},{"columnSpan":1,"name":"2"}]},{"marketTypes":["HCTG"],"columns":[{"columnSpan":1,"name":"OVER/UNDER"}]},{"marketTypes":["BTSC"],"columns":[{"columnSpan":1,"name":"GG/NG"}]}],"layout":2,"events":[{"stats":[{"url":"https://s5.sir.sportradar.com/stoiximan/en/match/41763007","providerId":1}],"sportId":"

In [5]:
df_teams = pd.DataFrame({'Home Team': HomeTeams, 'Away Team': AwayTeams})
df_values = pd.DataFrame(rows, columns=['1', 'x', '2', 'Over', 'Under', 'Yes', 'No'])
df_all = pd.concat([df_teams, df_values], axis=1)
df_all['Home Team'] = df_all['Home Team'].str.strip()
df_all['Away Team'] = df_all['Away Team'].str.strip()
display(df_all)

,Home Team,Away Team,1,x,2,Over,Under,Yes,No
0,Liverpool,Everton,1.38,5.30,7.50,2.07,1.75,1.65,2.15
1,Bournemouth,Wolverhampton Wanderers,2.42,3.40,2.90,1.82,2.00,1.62,2.20
2,Newcastle United,Crystal Palace,1.45,4.45,7.50,1.78,2.05,1.95,1.80
3,Manchester City,Brighton,1.44,4.90,6.70,1.85,1.98,1.47,2.57
4,Brentford,Burnley,1.72,3.85,4.70,1.82,2.00,1.75,2.00
5,Nottingham Forest,Luton,1.70,3.85,5.00,2.00,1.82,1.95,1.78
6,Chelsea,Arsenal,3.05,3.60,2.25,1.82,2.00,1.65,2.15
7,Sheffield United,Manchester United,6.40,4.60,1.47,1.60,2.35,1.72,2.05
8,Aston Villa,West Ham,1.98,3.70,3.60,1.60,2.35,1.50,2.45
9,Tottenham,Fulham,1.53,4.45,5.80,2.30,1.62,1.62,2.22


In [6]:
df = pd.read_csv('GatheredData/eplGMTStdTimeFiles/epl-2023-GMTStandardTime.csv')

display(df)

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,1,1,11/08/2023 20:00,Turf Moor,Burnley,Man City,NaN
1,2,1,12/08/2023 12:30,Emirates Stadium,Arsenal,Nottingham Forest,NaN
2,3,1,12/08/2023 15:00,Vitality Stadium,Bournemouth,West Ham,NaN
3,4,1,12/08/2023 15:00,Amex Stadium,Brighton,Luton,NaN
4,5,1,12/08/2023 15:00,Goodison Park,Everton,Fulham,NaN
...,...,...,...,...,...,...,...
375,376,38,19/05/2024 16:00,Selhurst Park,Crystal Palace,Aston Villa,NaN
376,377,38,19/05/2024 16:00,Anfield,Liverpool,Wolves,NaN
377,378,38,19/05/2024 16:00,Kenilworth Road,Luton,Fulham,NaN
378,379,38,19/05/2024 16:00,Etihad Stadium,Man City,West Ham,NaN


In [7]:
df_merged = pd.merge(df, df_all, on=['Home Team', 'Away Team'], how='right')
df_merged['Date']=Dates
display(df_merged)
df_merged.to_csv('UpcomingFixtures/Newfixtures.csv')

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result,1,x,2,Over,Under,Yes,No
0,85.0,9.0,21/10/2023,Anfield,Liverpool,Everton,NaN,1.38,5.30,7.50,2.07,1.75,1.65,2.15
1,NaN,NaN,21/10/2023,NaN,Bournemouth,Wolverhampton Wanderers,NaN,2.42,3.40,2.90,1.82,2.00,1.62,2.20
2,NaN,NaN,21/10/2023,NaN,Newcastle United,Crystal Palace,NaN,1.45,4.45,7.50,1.78,2.05,1.95,1.80
3,NaN,NaN,21/10/2023,NaN,Manchester City,Brighton,NaN,1.44,4.90,6.70,1.85,1.98,1.47,2.57
4,83.0,9.0,21/10/2023,Gtech Community Stadium,Brentford,Burnley,NaN,1.72,3.85,4.70,1.82,2.00,1.75,2.00
5,88.0,9.0,21/10/2023,The City Ground,Nottingham Forest,Luton,NaN,1.70,3.85,5.00,2.00,1.82,1.95,1.78
6,84.0,9.0,21/10/2023,Stamford Bridge,Chelsea,Arsenal,NaN,3.05,3.60,2.25,1.82,2.00,1.65,2.15
7,NaN,NaN,21/10/2023,NaN,Sheffield United,Manchester United,NaN,6.40,4.60,1.47,1.60,2.35,1.72,2.05
8,81.0,9.0,22/10/2023,Villa Park,Aston Villa,West Ham,NaN,1.98,3.70,3.60,1.60,2.35,1.50,2.45
9,NaN,NaN,23/10/2023,NaN,Tottenham,Fulham,NaN,1.53,4.45,5.80,2.30,1.62,1.62,2.22


In [8]:
#Downloading the updated results
import os
import requests

# URL of the file to download
url = "https://www.football-data.co.uk/mmz4281/2324/E0.csv"

# Name for the downloaded file
file_name = "E0-2023.csv"

# Path to the directory where you want to save the file
output_directory = "GatheredData/ExFiles"

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Full path for the downloaded file
output_path = os.path.join(output_directory, file_name)

# Download the file
response = requests.get(url)
if response.status_code == 200:
    with open(output_path, "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved as {output_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as GatheredData/ExFiles/E0-2023.csv


In [9]:
#Changing team names
temp_pd = pd.read_csv('GatheredData/ExFiles/E0-2023.csv')
team_dict = {
                     'Man City': 'Manchester City',
                     'Man United': 'Manchester United',
                     'Nott\'m Forest': 'Nottingham Forest',
                     'Newcastle': 'Newcastle United',
                     'Wolves': 'Wolverhampton Wanderers'
                         }
temp_pd['HomeTeam'] = temp_pd['HomeTeam'].replace(team_dict)
temp_pd['AwayTeam'] = temp_pd['AwayTeam'].replace(team_dict)

temp_pd.to_csv('GatheredData/ExFiles/E0-2023.csv')

In [10]:
# run the script named "my_script.py" with argument "arg1"
os.system("python ~/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewFixturesScraping2.py")
os.system("python ~/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewfixturesScraping2Month.py")
os.system("python ~/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewfixturesScraping2Month3.py")
os.system("python ~/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023Season.py")
os.system("python ~/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm.py")
os.system("python ~/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm_3Month.py")

21/10/2023
0            Liverpool
1          Bournemouth
2     Newcastle United
3      Manchester City
4            Brentford
5    Nottingham Forest
6              Chelsea
7     Sheffield United
Name: Home Team, dtype: object
0                    Everton
1    Wolverhampton Wanderers
2             Crystal Palace
3                   Brighton
4                    Burnley
5                      Luton
6                    Arsenal
7          Manchester United
Name: Away Team, dtype: object


/home/nickpad/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewFixturesScraping2.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewFixturesScraping2.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


22/10/2023
8    Aston Villa
Name: Home Team, dtype: object
8    West Ham
Name: Away Team, dtype: object
23/10/2023
9    Tottenham
Name: Home Team, dtype: object
9    Fulham
Name: Away Team, dtype: object


/home/nickpad/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewFixturesScraping2.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewFixturesScraping2.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewFixturesScraping2.py:54: SettingWithCopyWarning: 
A value is trying to be se

               HTeam  HM  HG  HGA  HPTS  ...  HPPDA  HOPPDA     HDC    HODC  HPTS/M
0          Liverpool   8  18    9    17  ...  10.70   15.17  11.000   7.375   2.125
1        Bournemouth   8   5   18     3  ...  11.00    6.89   8.375  12.750   0.375
2   Newcastle United   8  20    9    13  ...  10.77   13.81   7.250   5.625   1.625
3    Manchester City   8  17    6    18  ...  11.01   21.04  11.375   4.250   2.250
4          Brentford   8  11   12     7  ...   9.94   11.75   8.375   7.750   0.875
5  Nottingham Forest   8   8   10     9  ...  19.96   10.34   5.875   8.500   1.125
6            Chelsea   8  11    7    11  ...   8.09   18.67  12.250   5.375   1.375
7   Sheffield United   8   6   22     1  ...  15.72    8.07   3.500  12.125   0.125
8        Aston Villa   8  19   12    16  ...  15.51    9.69   9.375   7.375   2.000
9          Tottenham   8  18    8    20  ...   7.25   11.55  14.750   7.125   2.500

[10 rows x 12 columns]
                     ATeam  AM  AG  AGA  ...  AOPPDA

...

68    Fulham
Name: HomeTeam, dtype: object
68    Chelsea
Name: AwayTeam, dtype: object
0     Team  M  G  GA PTS  ...   PPDA  OPPDA        DC       ODC     PTS/M
12  Fulham  6  5  10   8  ...  14.72  14.16  6.333333  9.333333  1.333333

[1 rows x 12 columns]
0      Team  M  G GA PTS     xG   xGA  PPDA  OPPDA         DC  ODC     PTS/M
15  Chelsea  6  5  6   5  11.83  7.09  6.97  20.05  13.333333  5.5  0.833333
03/10/2023
2023-10-02
2023-10-02


/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm_3Month.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm_3Month.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


69    Luton
Name: HomeTeam, dtype: object
69    Burnley
Name: AwayTeam, dtype: object
0    Team  M  G  GA PTS    xG    xGA  PPDA OPPDA        DC        ODC     PTS/M
17  Luton  6  5  12   4  9.58  15.18  17.8  7.47  6.333333  10.166667  0.666667
0      Team  M  G  GA PTS    xG    xGA   PPDA  OPPDA   DC       ODC     PTS/M
19  Burnley  6  4  15   1  5.15  12.27  13.13  15.78  5.5  9.833333  0.166667
                      HTeam  HM  HG  ...        HDC       HODC    HPTS/M
0                   Burnley   0   0  ...   0.000000   0.000000  0.000000
1                   Arsenal   0   0  ...   0.000000   0.000000  0.000000
2               Bournemouth   0   0  ...   0.000000   0.000000  0.000000
3                  Brighton   0   0  ...   0.000000   0.000000  0.000000
4                   Everton   0   0  ...   0.000000   0.000000  0.000000
..                      ...  ..  ..  ...        ...        ...       ...
65  Wolverhampton Wanderers   6   6  ...   6.666667  13.500000  0.666667
66            

/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm_3Month.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm_3Month.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


0